In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import glob

path = r'.\Data' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

frame = pd.concat(li, axis=0, ignore_index=True)

In [3]:
frame['OP_CARRIER_AIRLINE_ID'].value_counts()

19393    1193096
19805     739343
19790     738063
20304     531153
19977     473780
20409     222323
20452     204580
19930     195276
20416     163993
20363     140708
20398     137974
20397     127018
20368     115899
20436     109178
20378     106872
20366      61598
19690      61059
19687      33301
Name: OP_CARRIER_AIRLINE_ID, dtype: int64

In [4]:
frame[frame['OP_CARRIER_AIRLINE_ID']==19393]['ORIGIN_AIRPORT_ID'].value_counts().head(20)

13232    72496
11292    70214
12889    65760
10821    63717
11259    56492
14107    53687
12191    44182
13204    37005
10693    36179
13796    34640
15016    34200
10397    33220
14679    31896
12892    30868
14831    27870
14893    26096
10423    24618
15304    23879
13198    19649
11697    18152
Name: ORIGIN_AIRPORT_ID, dtype: int64

In [5]:
frame[frame['OP_CARRIER_AIRLINE_ID']==19393]['DEST_AIRPORT_ID'].value_counts().head(20)

13232    72485
11292    70211
12889    65761
10821    63709
11259    56479
14107    53679
12191    44174
13204    36994
10693    36180
13796    34632
15016    34200
10397    33221
14679    31897
12892    30873
14831    27872
14893    26106
10423    24616
15304    23875
13198    19651
11697    18148
Name: DEST_AIRPORT_ID, dtype: int64

In [6]:
busiest_airline = frame[frame['OP_CARRIER_AIRLINE_ID']==19393]
filtered_dataframe = busiest_airline[(busiest_airline['ORIGIN_AIRPORT_ID']==13232) | (busiest_airline['ORIGIN_AIRPORT_ID']==11292) | (busiest_airline['ORIGIN_AIRPORT_ID']==12889) | (busiest_airline['DEST_AIRPORT_ID']==13232) | (busiest_airline['DEST_AIRPORT_ID']==11292) | (busiest_airline['DEST_AIRPORT_ID']==12889)]

In [7]:
filtered_dataframe

,YEAR,QUARTER,FL_DATE,OP_CARRIER_AIRLINE_ID,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,DEP_TIME,DEP_DELAY,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY
24037,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,1348.0,13.0,1454.0,9.0,0.0,NaN,NaN,NaN,NaN,NaN
24038,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,2038.0,28.0,2147.0,22.0,0.0,NaN,0.0,0.0,0.0,0.0
24039,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,543.0,-2.0,648.0,-12.0,0.0,NaN,NaN,NaN,NaN,NaN
24040,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,1058.0,23.0,1201.0,11.0,0.0,NaN,NaN,NaN,NaN,NaN
24041,2018,2,4/4/2018 12:00:00 AM,19393,10423,11292,1743.0,-2.0,1856.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5355194,2022,3,7/13/2022 12:00:00 AM,19393,11638,11292,1458.0,3.0,1811.0,1.0,0.0,NaN,NaN,NaN,NaN,NaN
5355195,2022,3,7/13/2022 12:00:00 AM,19393,11638,12889,1544.0,-6.0,1652.0,-8.0,0.0,NaN,NaN,NaN,NaN,NaN
5355196,2022,3,7/13/2022 12:00:00 AM,19393,11638,12889,1226.0,71.0,1343.0,78.0,0.0,NaN,0.0,0.0,7.0,0.0
5355197,2022,3,7/13/2022 12:00:00 AM,19393,11638,12889,549.0,-6.0,654.0,-6.0,0.0,NaN,NaN,NaN,NaN,NaN
